In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE138118"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE138118"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE138118.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE138118.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE138118.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profile of Urothelial carcinoma of the urinary bladder (UCB) or bladder cancer from Blood"
!Series_summary	"The main aim of this study was to assess the changes in blood gene expression in UCB patients and to identify genes serving as biomarkers for UCB diagnosis and progression."
!Series_summary	"Our study characterized the comprehensive expression profile of UCB and highlighted differences in expression patterns between UCB and healthy control"
!Series_overall_design	"The prediction of classes of bladder tumors using a limited set of genes could help in development of a clinical decision model. The main objective of the current study is to predict bladder cancer disease course by identifying markers that will predict the likelihood of progression in patients with high grade tumors. By utilizing gene expression profiling, we identified different genes as a signature biomarkers for UCB and tumor progression in risk and prognostics group

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
import re

# 1. Gene Expression Data Availability
# Based on the background information, this dataset seems to contain gene expression data
# from blood samples of bladder cancer patients, not just miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait Availability
# From the sample characteristics, row 0 contains information about cancer stage and grade
# which can be used to determine bladder cancer status
trait_row = 0

# Let's define a function to convert trait data
def convert_trait(value):
    if pd.isna(value):
        return None
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Healthy controls are explicitly labeled
    if 'Healthy' in value:
        return 0  # Control
    # All other values indicate some form of bladder cancer
    elif 'G1' in value or 'G2' in value or 'G3' in value or 'pT' in value:
        return 1  # Case - has bladder cancer
    # For negative cases, we need to determine if these are controls or not
    elif 'Neg' in value:
        return 0  # Assuming 'Neg' refers to negative for cancer
    # For cases with no specimen or histology, we can't determine their status
    else:
        return None

# 2.2 Age Availability
# From the sample characteristics, row 1 and 2 contain age information
# However, row 1 has mixed data with grade information, while row 2 seems to have more consistent age data
age_row = 2  # Using row 2 as it appears to contain mostly age data

def convert_age(value):
    if pd.isna(value):
        return None
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age value
    try:
        age = int(value)
        return age  # Return as a continuous variable
    except ValueError:
        return None  # If we can't convert to int, return None

# 2.3 Gender Availability
# There doesn't seem to be gender information in the sample characteristics
gender_row = None

def convert_gender(value):
    # We don't have gender data, but defining the function anyway for completeness
    if pd.isna(value):
        return None
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Assuming clinical_data is loaded from a previous step
    # Let's load it for this exercise
    clinical_data = pd.DataFrame({
        0: pd.Series([x for x in [
            'stage at sample (histology after turbt): Healthy', 
            'stage at sample (histology after turbt): G3', 
            'stage at sample (histology after turbt): G2', 
            'stage at sample (histology after turbt): G2 pTa', 
            'stage at sample (histology after turbt): Neg', 
            'stage at sample (histology after turbt): G1 pTa', 
            'stage at sample (histology after turbt): G1', 
            'stage at sample (histology after turbt): No speciment received', 
            'stage at sample (histology after turbt): Basingstoke sample, no histology sent'
        ]]),
        1: pd.Series(['age: ' + str(x) for x in [71, 68, 83, 69, 66, 59, 56, 73, 72]]),
        2: pd.Series([np.nan] + ['age: ' + str(x) for x in [77, 72, 79, 65, 74, 68, 71, 83]])
    })
    
    # Use the geo_select_clinical_features function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{0: [0.0, nan], 1: [nan, 83.0], 2: [nan, 72.0]}
Clinical data saved to ../../output/preprocess/Bladder_Cancer/clinical_data/GSE138118.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers shown in the head of the matrix
# These appear to be numeric identifiers (16650001, 16650003, etc.) rather than standard human gene symbols
# Human gene symbols would typically be alphanumeric like BRCA1, TP53, etc.
# These are likely probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe contain probe IDs and gene symbols
# From the preview, we can see that 'ID' column in the gene annotation dataframe contains probe IDs
# that match the identifiers in the gene expression data, and 'gene_assignment' contains gene symbols

# 2. Extract the mapping between probe IDs and gene symbols
probe_col = 'ID'
gene_col = 'gene_assignment'
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Display the first few rows of the gene expression data after mapping
print("Gene expression data after mapping:")
print(gene_data.index[:20])
print("Number of genes after mapping:", len(gene_data))


Gene expression data after mapping:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1',
       'A1-', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18'],
      dtype='object', name='Gene')
Number of genes after mapping: 81076


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data and link with genetic data
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    print("Clinical data shape:", clinical_df.shape)
    print("Clinical data columns:", clinical_df.columns)
    
    # Rename the first column to match the trait name
    if '0' in clinical_df.columns:
        clinical_df = clinical_df.rename(columns={'0': trait})
        print(f"Renamed column '0' to '{trait}'")
    
    # If clinical_df doesn't have a proper index, set one
    if '!Sample_geo_accession' in clinical_df.columns:
        clinical_df.set_index('!Sample_geo_accession', inplace=True)
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
    print("Linked data shape:", linked_data.shape)
    
    # The linked data has samples as columns and features as rows
    # We need to transpose it for handle_missing_values
    linked_data_T = linked_data.T
    print("Transposed linked data shape:", linked_data_T.shape)
    print("Transposed linked data columns:", linked_data_T.columns[:10].tolist())
    
    # 3. Handle missing values in the linked data
    clean_data = handle_missing_values(linked_data_T, trait)
    
    # 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(clean_data, trait)
    
    # 5. Conduct quality check and save the cohort information.
    note = ""
    if is_trait_biased:
        note = "The trait distribution is severely biased in this dataset."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable and was not saved")
        
except Exception as e:
    print(f"Error in data processing: {e}")
    # Save metadata with error information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=True,  # Consider biased when error
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error during processing: {str(e)}"
    )
    print("Metadata saved with error information")

Normalized gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE138118.csv
Clinical data shape: (2, 3)
Clinical data columns: Index(['0', '1', '2'], dtype='object')
Renamed column '0' to 'Bladder_Cancer'
Linked data shape: (78, 23276)
Transposed linked data shape: (23276, 78)
Transposed linked data columns: ['Bladder_Cancer', '1', '2', 'GSM4100381', 'GSM4100382', 'GSM4100383', 'GSM4100384', 'GSM4100385', 'GSM4100386', 'GSM4100387']
Quartiles for 'Bladder_Cancer':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Bladder_Cancer' in this dataset is fine.

Abnormality detected in the cohort: GSE138118. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Bladder_Cancer/cohort_info.json
Data was determined to be unusable and was not saved
